In [9]:
import numpy as np
import os

# plotting 
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid

# data manager and analysis
import vodex as vx
import numan as nu

# writing files
import tifffile as tif

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Project structure: 

Provide the project folder, then the "processed" folder will be created inside. 

As you keep going with the analysis, the folder will have the following structure: 

```
processed
│   experiment.json <----------------------------------- (WILL BE DONE in this notebook) the file that contains everything about the experiment, you are creating it once and will be reusing ever after
│   experiment_dff.json <------------------------------- (WILL BE DONE in this notebook) everything about the experiment, but loads from the dff movie, not from the raw data
└───dff_movie  <---------------------------------------- (WILL BE DONE in this notebook) the dff movie :) 
│   │   dff_movie_0000.tif
│   │   dff_movie_0001.tif
│   │   ... 
└───tscore_volumes  <----------------------------------- t-score tif files per pair
│   └───2v3
│       │   tscore_2v3.tif
│   └───3v5
│       │   tscore_3v5.tif
│   └───2v5
│       │   tscore_2v5.tif
│   └───2vB
│       │   tscore_2vB.tif
│   └───3vB
│       │   tscore_3vB.tif
│   └───5vB
│       │   tscore_5vB.tif
│   └───BvB
│       │   tscore_BvB1.tif
│       │   tscore_BvB2.tif
│       │   tscore_BvB3.tif
│       │   tscore_BvB4.tif
│       │   tscore_BvB5.tif
└───diff_volumes  <------------------------------------- absolute difference tif files per pair
│   └───2v3
│       │   diff_2v3.tif
│   └───3v5
│       │   diff_3v5.tif
│   └───...
└───spots
│   └───imaris  <--------------------------------------- ATTENTION : You need to put stuff generated by imaris into this folder!!! 
│       │   └───tscore_2v3_Statistics
│       │       │     tscore_2v3_Position.csv
│       │       │     tscore_2v3_Diameter.csv
│       │       │     ...
│       │   └───tscore_3v5_Statistics
│       │       │     tscore_3v5_Position.csv
│       │       │     tscore_3v5_Diameter.csv
│       │       │     ...
│       │   └───tscore_2v5_Statistics
│       │       │     ...
│       │     ...
│   └───signals  <-------------------------------------- json files with the extracted signals, also will have the group info after you added it
│       │   spots_2v3.json
│       │   spots_3v5.json
│       │   spots_2v5.json
│       │     ...
└───reports  <------------------------------------------ tiffs and pdf with the cells and signals found 
│   └───all_significant  <------------------------------ tiffs and pdf with all significant cells per group
│       │   └───signals  <------------------------------ pdfs with signals
│       │       │     significant_2v3.pdf
│       │       │     significant_3v5.pdf
│       │       │     ...
│       │   └───images <-- tif masks 
│       │       │     significant_2v3.tif
│       │       │     significant_3v5.tif
│       │       │     ...
│   └───groupped  <------------------------------------- tiffs and pdf where the cells are groupped based on signal shape .. or anything else you want
│       │   readme.txt  <------------------------------- ATTENTION : you need to describe the groups
│       │   └───signals  <------------------------------ pdfs with signals
│       │       │     shape1_2v3.pdf
│       │       │     shape2_2v3.pdf
│       │       │     shape3_2v3.pdf
│       │       │     ...
│       │       │     shape1_3v5.pdf
│       │       │     shape2_3v5.pdf
│       │       │     shape3_3v5.pdf
│       │       │     ...
│       │   └───images  <------------------------------ tif masks 
│       │       │     shape1_2v3.tif
│       │       │     shape2_2v3.tif
│       │       │     shape3_2v3.tif
│       │       │     ...
│       │       │     shape1_3v5.tif
│       │       │     shape2_3v5.tif
│       │       │     shape3_3v5.tif
│       │       │     ...
│       │     ...
```
# Set project folder
Also, if the processed folder exists, it will complain ...

In [ ]:
project_folder = "D:/Code/repos/numan/notebooks/data/2vs3vs5/"

path = os.path.join(project_folder, 'processed')
assert not os.path.isdir(path), 'the directory "processed" already exists in the project,' \
                                ' did you mean to process another project?'
os.makedirs(path)

We will also set the processed directory as our working directory, this step is important , since all the paths later are relative to this folder. Verify that the output of the cell is the "processed" folder inside your project folder. 

In [18]:
os.chdir(path)
os.getcwd()

'D:\\Code\\repos\\numan\\notebooks\\data\\2vs3vs5\\processed'

# Define experiment: 

In [19]:
blank = vx.Condition(vx.Stimulus('blank','blank'), name = 'blank')
dot2 = vx.Condition(vx.Stimulus('2dot','2dot'), name = 'dot2')
dot3 = vx.Condition(vx.Stimulus('3dot','3dot'), name = 'dot3')
dot5 = vx.Condition(vx.Stimulus('5dot','5dot'), name = 'dot5')

spec = {}
spec['conditions'] = [blank,dot3,blank,dot2,blank,dot5,blank,dot2,blank,dot3,blank,
                    dot2,blank,dot5,blank,dot3,blank,dot5]
spec['timing'] = np.array([7,1,5,1,6,1,6,1,9,1,5,1,9,1,6,1,9,1])*52 # in frames
spec['project_dir'] = "D:/Code/repos/numan/notebooks/data/2vs3vs5/raw/"
spec['frames_per_volume'] = 52

experiment = vx.Experiment.from_spec(spec)
experiment.summary()

Total of 8 files.
Check the order :
[ 0 ] 20220421_ok08_abtl_h2bcamp6s_9dpf_2v3v5_2P_1_MMStack_Pos0.ome.tif : 8910 frames
[ 1 ] 20220421_ok08_abtl_h2bcamp6s_9dpf_2v3v5_2P_1_MMStack_Pos0_1.ome.tif : 8909 frames
[ 2 ] 20220421_ok08_abtl_h2bcamp6s_9dpf_2v3v5_2P_1_MMStack_Pos0_2.ome.tif : 8909 frames
[ 3 ] 20220421_ok08_abtl_h2bcamp6s_9dpf_2v3v5_2P_1_MMStack_Pos0_3.ome.tif : 8909 frames
[ 4 ] 20220421_ok08_abtl_h2bcamp6s_9dpf_2v3v5_2P_1_MMStack_Pos0_4.ome.tif : 8909 frames
[ 5 ] 20220421_ok08_abtl_h2bcamp6s_9dpf_2v3v5_2P_1_MMStack_Pos0_5.ome.tif : 8909 frames
[ 6 ] 20220421_ok08_abtl_h2bcamp6s_9dpf_2v3v5_2P_1_MMStack_Pos0_6.ome.tif : 8909 frames
[ 7 ] 20220421_ok08_abtl_h2bcamp6s_9dpf_2v3v5_2P_1_MMStack_Pos0_7.ome.tif : 4092 frames

Cycle length: 3692
Condition ['blank']: for 364 frames
Condition ['3dot']: for 52 frames
Condition ['blank']: for 260 frames
Condition ['2dot']: for 52 frames
Condition ['blank']: for 312 frames
Condition ['5dot']: for 52 frames
Condition ['blank']: for 312 fra

**Make sure everything above is correct** and then , if it is , save it for future use: 

In [20]:
vx.to_json(experiment, 'experiment.json')

# Create dff movie

In [ ]:
batch_size = 25 # in volumes
window_size = 15 # in volumes

assert not os.path.isdir('dff_movie'), "the directory 'dff_movie' already exists in the project," \
                                " You will be overwriting it's content, if you proceed. Did you mean to process another project?"
os.makedirs('dff_movie')

In [ ]:
nu.Preprocess(experiment).batch_dff('dff_movie',batch_size, window_size, verbose=True )

100%|█████████████████████████████████████████████████████████████████████████████| 1300/1300 [00:03<00:00, 360.50it/s]


written frames : 0 - 17, out of 1278


100%|█████████████████████████████████████████████████████████████████████████████| 1300/1300 [00:03<00:00, 348.52it/s]


### Now set the dff_movie folder as the data to process : 

In [10]:
spec['project_dir'] = 'dff_movie/'
experiment_dff = vx.Experiment.from_spec(spec)
experiment_dff.summary()

Total of 115 files.
Check the order :
[ 0 ] dff_movie_0000.tif : 936 frames
[ 1 ] dff_movie_0001.tif : 572 frames
[ 2 ] dff_movie_0002.tif : 572 frames
[ 3 ] dff_movie_0003.tif : 572 frames
[ 4 ] dff_movie_0004.tif : 572 frames
[ 5 ] dff_movie_0005.tif : 572 frames
[ 6 ] dff_movie_0006.tif : 572 frames
[ 7 ] dff_movie_0007.tif : 572 frames
[ 8 ] dff_movie_0008.tif : 572 frames
[ 9 ] dff_movie_0009.tif : 572 frames
[ 10 ] dff_movie_0010.tif : 572 frames
[ 11 ] dff_movie_0011.tif : 572 frames
[ 12 ] dff_movie_0012.tif : 572 frames
[ 13 ] dff_movie_0013.tif : 572 frames
[ 14 ] dff_movie_0014.tif : 572 frames
[ 15 ] dff_movie_0015.tif : 572 frames
[ 16 ] dff_movie_0016.tif : 572 frames
[ 17 ] dff_movie_0017.tif : 572 frames
[ 18 ] dff_movie_0018.tif : 572 frames
[ 19 ] dff_movie_0019.tif : 572 frames
[ 20 ] dff_movie_0020.tif : 572 frames
[ 21 ] dff_movie_0021.tif : 572 frames
[ 22 ] dff_movie_0022.tif : 572 frames
[ 23 ] dff_movie_0023.tif : 572 frames
[ 24 ] dff_movie_0024.tif : 572 fram

**Make sure everything except the filenames looks identical to the experiment we defined at the beginning** and then , if it does , save it for future use: 

In [11]:
assert experiment.n_frames == experiment_dff.n_frames, "The number of frames in the experiment and experiment_dff is different! Did you not finish processing the dff movie? "

vx.to_json(experiment_dff, 'experiment_dff.json')